## **Adhar Data Analysis**

### Aim
    Create a dataframe with Total Aadhaar's generated for each state
    Create a dataframe with Total Aadhaar's generated by each enrollment agency
    Create dataframe with top 10 districts with maximum Aadhaar's generated for both Male and Female?
    Create a dataframe with Total Aadhaar's generated for top 10 least state
    For which age most adhar card has declined ?

In [1]:
pip install pyspark # installing pyspark enviornment 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 199 kB 26.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=3dc42b6a2ffbce5324076791d7e0473fdce0fef5e3a3c15d273c8e5264efb11e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [5]:
# importing required libraries
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,SparkSession
from pyspark.sql import functions as f

In [7]:
#setting the session for pypark
conf=SparkConf().setAppName('UDI')
sc=SparkContext(conf=conf)
spark=SparkSession.builder.appName('UDI_analysis').getOrCreate()
sqlcont=SQLContext(spark)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [9]:
# loading data into a spark dataframe
dfUDI=spark.read.csv('/content/UIDAI-ENR-DETAIL-20170308.csv',header=True)

+--------------------+--------------------+-------------+----------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|           Registrar|    Enrolment Agency|        State|  District|Sub District|Pin Code|Gender|Age|Aadhaar generated|Enrolment Rejected|Residents providing email|Residents providing mobile number|
+--------------------+--------------------+-------------+----------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|      Allahabad Bank|A-Onerealtors Pvt...|Uttar Pradesh| Allahabad|        Meja|  212303|     F|  7|                1|                 0|                        0|                                1|
|      Allahabad Bank|Asha Security Gua...|Uttar Pradesh| Sonbhadra| Robertsganj|  231213|     M|  8|                1|                 0|                        0|                                0|
|    

### Show a sample of 5 records from dataset.

In [11]:
dfUDI.show(5) # displaying first 5 rows 

+--------------+--------------------+-------------+---------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|     Registrar|    Enrolment Agency|        State| District|Sub District|Pin Code|Gender|Age|Aadhaar generated|Enrolment Rejected|Residents providing email|Residents providing mobile number|
+--------------+--------------------+-------------+---------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|Allahabad Bank|A-Onerealtors Pvt...|Uttar Pradesh|Allahabad|        Meja|  212303|     F|  7|                1|                 0|                        0|                                1|
|Allahabad Bank|Asha Security Gua...|Uttar Pradesh|Sonbhadra| Robertsganj|  231213|     M|  8|                1|                 0|                        0|                                0|
|Allahabad Bank|   SGS INDIA PVT LTD|Utt

#### Shape of DataSet

In [33]:
# countring rows and features
row= dfUDI.count()
col=len(dfUDI.columns)
print('shape of dataframe: (',row,',',col,')') 

# or we can check the shape by converting spark dataset into pandas
print('shape of dataframe by pandas: ',  dfUDI.toPandas().shape,)

shape of dataframe: ( 82154 , 12 )
shape of dataframe by pandas:  (82154, 12)


#### Feature Renaming

In [ ]:
# feature renaming 
for col in dfUDI.columns:
  dfUDI=dfUDI.withColumnRenamed(col,col.replace(' ','_')) # replacing space with _


###     Read the data with data types.

In [34]:
dfUDI.printSchema() # printing schema

root
 |-- Registrar: string (nullable = true)
 |-- Enrolment_Agency: string (nullable = true)
 |-- State: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Sub_District: string (nullable = true)
 |-- Pin_Code: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Aadhaar_generated: string (nullable = true)
 |-- Enrolment_Rejected: string (nullable = true)
 |-- Residents_providing_email: string (nullable = true)
 |-- Residents_providing_mobile_number: string (nullable = true)



#### Create a dataframe with Total Aadhaar's generated for each state

In [41]:
df_total_adhar=dfUDI.groupBy("State").count().orderBy(f.desc('count')).withColumnRenamed('count','Total Adhar')
# grouping by State and counting records, sorted data into descending order and renaming the count column
df_total_adhar.show(50) # showing top 50 recods

+--------------------+-----------+
|               State|Total Adhar|
+--------------------+-----------+
|               Bihar|      28655|
|         West Bengal|      11435|
|           Rajasthan|       8585|
|       Uttar Pradesh|       7836|
|        Chhattisgarh|       4322|
|         Maharashtra|       4249|
|      Madhya Pradesh|       3322|
|              Punjab|       2292|
|          Tamil Nadu|       2193|
|           Karnataka|       2161|
|               Delhi|       1358|
|           Jharkhand|       1148|
|               Assam|        915|
|      Andhra Pradesh|        824|
|             Gujarat|        783|
|              Odisha|        607|
|             Haryana|        596|
|           Telangana|        317|
|   Jammu and Kashmir|        155|
|   Arunachal Pradesh|        138|
|         Uttarakhand|        118|
|          Chandigarh|         40|
|              Kerala|         22|
|             Mizoram|         16|
|    Himachal Pradesh|         12|
|            Nagalan

#### Create a dataframe with Total Aadhaar's generated by each enrollment agency

In [42]:
total_Adhar_enroll=dfUDI.groupBy('Enrolment_Agency').count().orderBy(f.desc('count')).withColumnRenamed('count','Total Adhar')
#grouping dataframe using Enrolment_Agency and counting the records.
#sorted data into descending order and after that renaming the column to Total Adhar
total_Adhar_enroll.show(50) #showing top 50 records

+--------------------+-----------+
|    Enrolment_Agency|Total Adhar|
+--------------------+-----------+
|             CSC SPV|      18862|
|SREI INFRASTRUCTU...|      16972|
|AKSH OPTIFIBRE LI...|      12579|
|   CMS Computers Ltd|       9170|
|               BASIX|       5916|
|A I Soc for Elect...|       4058|
|               CHIPS|       3856|
|  Amar Constructions|       2694|
|VEETECHNOLOGIES P...|       2428|
|Atalji Janasnehi ...|       1458|
|Bloom Solutions P...|        554|
|PROTEX COMPUTER P...|        509|
| Agro Tech Engineers|        473|
|Academy of Manage...|        471|
|OSWAL COMPUTERS &...|        460|
|Twinstar Industri...|        361|
|    APOnline Limited|        278|
|AVVAS INFOTECH PV...|        264|
|CALANCE SOFTWARE ...|        214|
|Asha Security Gua...|        161|
|Frontech Systems ...|        105|
|      Binary Systems|         90|
|CHESSY CONSULTANT...|         86|
|Abha Systems And ...|         73|
|     Alankit Limited|         23|
|DATASOFT COMPUTER..

#### Create dataframe with top 10 districts with maximum Aadhaar's generated for both Male and Female?

In [47]:
df_Female=dfUDI.filter(dfUDI.Gender=="F") # data filter for gender Female and storing data into seprate dataframe
df_Female.groupBy("District").count().orderBy(f.desc('count')).withColumnRenamed('count','Total Adhar Female').show(10)
# grouping dataframe by district and counting records
# sorting data in descending order and renaming count column into Toatl Adhar Female
# showing top 10 records

+-----------------+------------------+
|         District|Total Adhar Female|
+-----------------+------------------+
|             Gaya|               556|
|            Nadia|               501|
|            Patna|               481|
|South 24 Parganas|               477|
|        Bhagalpur|               476|
|          Katihar|               464|
|North 24 Parganas|               405|
|   East Champaran|               360|
|         Khagaria|               338|
|           Jaipur|               338|
+-----------------+------------------+
only showing top 10 rows



In [48]:
df_Male=dfUDI.filter(dfUDI.Gender=="M") # data filter for gender Male abd storing data into seprate dataframe
df_Male.groupBy("District").count().orderBy(f.desc('count')).withColumnRenamed('count','Total Adhar Male').show(10)
# grouping dataframe by district and counting records
# sorting data in descending order and renaming count column into Toatl Adhar Male
# showing top 10 records

+-----------------+----------------+
|         District|Total Adhar Male|
+-----------------+----------------+
|             Gaya|            1825|
|        Bhagalpur|            1695|
|          Katihar|            1338|
|   East Champaran|            1253|
|            Nadia|            1241|
|        Madhubani|            1125|
|            Patna|            1085|
|          Nalanda|            1062|
|           Munger|            1030|
|South 24 Parganas|             951|
+-----------------+----------------+
only showing top 10 rows



#### Create a dataframe with Total Aadhaar's generated for top 10 least state

In [52]:
df_top_10_state=dfUDI.groupBy('State').count().orderBy(f.desc('count')).withColumnRenamed('count','Total Adhar')
#grouping by state and counting records
#sorting in descending, renaming the count column in total adhar
df_top_10_state.show(10) # showing top 10 states which genrated adhar

+--------------+-----------+
|         State|Total Adhar|
+--------------+-----------+
|         Bihar|      28655|
|   West Bengal|      11435|
|     Rajasthan|       8585|
| Uttar Pradesh|       7836|
|  Chhattisgarh|       4322|
|   Maharashtra|       4249|
|Madhya Pradesh|       3322|
|        Punjab|       2292|
|    Tamil Nadu|       2193|
|     Karnataka|       2161|
+--------------+-----------+
only showing top 10 rows



#### For which age most adhar card has declined ?

In [56]:
df_rejected=dfUDI.filter(dfUDI.Enrolment_Rejected==1) # data filtered for enrollment rejection  and saving data into new dataframe
df_rejected.groupBy('Age').count().orderBy(f.desc('count')).withColumnRenamed('count','Total Rejected Count').show(1)
# grouping data with age and counting records, safter sorting in descending and renaming the count column
#showing top age record for which most of the adhar declines

+---+--------------------+
|Age|Total Rejected Count|
+---+--------------------+
|  4|                 564|
+---+--------------------+
only showing top 1 row

